<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/without-finetuning/twadr_9_pretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
curl: (60) SSL certificate problem: certificate has expired
More details here: https://curl.haxx.se/docs/sslcerts.html

curl failed to verify the legitimacy of the server and therefore could not
establish a secure connection to it. To learn more about this situation and
how to fix it, please visit the web page mentioned above.


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'TwADR-L.fold-9.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-9.validation.csv",header=None)
valid.head()

,0,1
0,13,up all night
1,13,insomnia
2,60,cigarette withdrawal
3,85,tremor
4,121,itches


In [12]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-9.test.csv",header=None)
test.head()

,0,1
0,62,feeling proper psycho
1,74,useless without
2,85,tremor
3,91,jitters
4,97,screwing with my short term memory


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_asapatient_twadr_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'pain',
 'lipitor',
 'in',
 'of',
 'muscle',
 'severe',
 'my',
 'to',
 'myalgia',
 'fatigue',
 'loss',
 'and',
 'lower',
 '-',
 'limb',
 'cramp',
 'weakness',
 'depression',
 'stomach',
 'the',
 'back',
 'cramps',
 'increased',
 'joint',
 'memory',
 'upper',
 'gas',
 'leg',
 'mood',
 'headache',
 'legs',
 'pains',
 'insomnia',
 'shoulder',
 'disorder',
 'feeling',
 'unable',
 'anxiety',
 'knee',
 ',',
 'arthralgia',
 'neck',
 'foot',
 'excessive',
 'like',
 'nausea',
 'weight',
 'gastrointestinal',
 'numbness',
 'dizziness',
 'heart',
 'arthritis',
 'aches',
 'abnormal',
 'disease',
 'hip',
 'abdominal',
 'sleep',
 'tired',
 'skin',
 'swelling',
 '/',
 'bleeding',
 'me',
 'arthrotec',
 'difficulty',
 ')',
 'walk',
 'asthenia',
 '(',
 'a',
 'blood',
 'walking',
 'problems',
 'stiffness',
 'chest',
 'ache',
 'feet',
 'vision',
 'liver',
 'i',
 'withdrawal',
 'hand',
 'syndrome',
 'muscles',
 'hair'

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
# learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [0]:
# learn.fit_one_cycle(1, 1e-2)

In [0]:
# learn.fit_one_cycle(1, 1e-2)

In [0]:
# learn.fit_one_cycle(1, 1e-3)

In [0]:
# learn.fit_one_cycle(1, 1e-3)

In [0]:
# learn.save_encoder('bigtweet_fold0_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
# learn.load_encoder('bigtweet_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.731974,7.264914,0.000000,00:22


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,7.463454,6.720620,0.058333,00:26


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,7.104456,7.990450,0.016667,01:10


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.811532,12.912779,0.025000,01:07
1,6.597442,9.696716,0.025000,01:09


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.231049,12.823854,0.016667,01:11
1,6.029032,11.062865,0.025000,01:07


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.768661,16.538206,0.000000,01:08
1,5.587334,11.080050,0.025000,01:12


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.375212,11.821712,0.025000,01:12
1,5.212023,19.244080,0.025000,01:05


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.053947,12.416724,0.008333,01:10
1,4.912755,8.865571,0.008333,01:13


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.719159,12.628475,0.008333,01:05
1,4.697905,12.453599,0.008333,01:09
2,4.507733,9.168921,0.033333,01:11
3,4.318503,11.787702,0.016667,01:10


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.195561,18.303658,0.000000,01:07
1,4.132877,24.540535,0.000000,01:07
2,3.981174,7.505272,0.083333,01:10
3,3.830514,10.471571,0.008333,01:08


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.658526,7.546276,0.025000,01:10
1,3.613602,11.514052,0.008333,01:09
2,3.446274,9.140631,0.041667,01:06
3,3.303524,7.404729,0.083333,01:10


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.145449,12.999233,0.033333,01:08
1,3.173433,7.316425,0.041667,01:09
2,3.006764,13.634416,0.000000,01:09
3,2.837898,15.371143,0.000000,01:12


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.704866,20.690094,0.016667,01:05
1,2.688977,7.708551,0.083333,01:10
2,2.527274,12.793599,0.016667,01:06
3,2.392092,10.803349,0.041667,01:12


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.274287,10.282871,0.058333,01:08
1,2.251254,9.051082,0.041667,01:08
2,2.136079,16.974464,0.008333,01:05
3,1.973526,11.200023,0.025000,01:07


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.855630,15.474652,0.016667,01:07
1,1.888893,6.029688,0.175000,01:11
2,1.754389,8.932000,0.166667,01:05
3,1.620500,7.735482,0.158333,01:08


In [41]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.514748,8.269018,0.133333,01:09
1,1.572816,7.768852,0.125000,01:09
2,1.459449,7.869627,0.091667,01:11
3,1.323045,9.334691,0.083333,01:08


In [42]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.225661,7.640219,0.125000,01:09
1,1.248015,10.486991,0.091667,01:05
2,1.190797,7.219748,0.141667,01:09
3,1.084929,7.598272,0.125000,01:09


In [43]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.985707,8.096421,0.091667,01:12
1,1.024006,7.247991,0.183333,01:10
2,0.982364,5.409455,0.233333,01:09
3,0.878785,6.195228,0.166667,01:09


In [44]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.790935,6.128771,0.200000,01:08
1,0.840584,6.648472,0.125000,01:10
2,0.806822,6.469163,0.191667,01:09
3,0.719804,7.553429,0.100000,01:07


In [45]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.685786,7.551909,0.125000,01:08
1,0.728247,6.136492,0.283333,01:12
2,0.708060,5.764902,0.266667,01:07
3,0.638129,5.449169,0.266667,01:12


In [46]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.564166,5.323936,0.308333,01:10
1,0.664967,7.807470,0.125000,01:10
2,0.628877,7.498668,0.133333,01:07
3,0.580487,5.862264,0.233333,01:11


In [47]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.529013,6.036341,0.291667,01:10
1,0.541003,5.564542,0.316667,01:07
2,0.542532,5.156891,0.383333,01:12
3,0.500837,6.299444,0.258333,01:08


In [48]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.453477,6.283143,0.208333,01:06
1,0.521050,4.961279,0.391667,01:12
2,0.517263,6.602246,0.275000,01:09
3,0.464595,7.069245,0.250000,01:09


In [49]:
learn.fit_one_cycle(2, slice(2e-3/1000, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.427627,5.674899,0.283333,01:12
1,0.421455,5.825258,0.258333,01:10


In [50]:
learn.fit_one_cycle(1, slice(2e-3/1000, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.446505,5.762633,0.333333,01:10


In [51]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.410193,5.233203,0.408333,01:11
1,0.469404,4.733602,0.433333,01:07
2,0.485507,5.929931,0.325000,01:10
3,0.441161,5.583977,0.283333,01:12


In [52]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.372606,6.651093,0.258333,01:06
1,0.460464,5.695323,0.308333,01:12
2,0.454004,6.143911,0.250000,01:07
3,0.439201,6.435668,0.241667,01:10


In [53]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.391669,6.183679,0.275000,01:08
1,0.397907,5.132690,0.441667,01:09
2,0.405651,5.577615,0.391667,01:12
3,0.402875,6.759942,0.225000,01:06


In [54]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.366424,5.533055,0.366667,01:07
1,0.440313,5.390581,0.358333,01:08
2,0.442399,5.412545,0.441667,01:12
3,0.382771,5.838575,0.391667,01:05


In [55]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.379093,7.120891,0.258333,01:06
1,0.421649,6.866512,0.233333,01:12
2,0.418411,5.882223,0.325000,01:11
3,0.379111,5.689028,0.358333,01:13


In [61]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.380457,6.447822,0.291667,01:07
1,0.403855,5.025466,0.458333,01:10
2,0.398992,5.111796,0.441667,01:12
3,0.373021,5.247550,0.433333,01:08


In [66]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.392785,5.338969,0.391667,01:06
1,0.402035,5.474401,0.333333,01:09


In [71]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.388224,5.390196,0.333333,01:11
1,0.388489,5.798946,0.275000,01:05
2,0.379026,5.373238,0.341667,01:08
3,0.360218,5.548993,0.325000,01:12


In [72]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.327328,5.307390,0.341667,01:06
1,0.376470,6.260763,0.191667,01:10
2,0.394545,4.928537,0.391667,01:10
3,0.344215,5.097098,0.358333,01:13


In [73]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.348470,5.376241,0.358333,01:10
1,0.353904,5.215277,0.408333,01:12
2,0.357935,4.993566,0.416667,01:12
3,0.346627,5.148569,0.416667,01:09


In [74]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.347862,5.194462,0.341667,01:07
1,0.381477,5.626313,0.333333,01:05
2,0.382481,5.166894,0.375000,01:07
3,0.354709,4.908910,0.475000,01:11


In [0]:
# save the best model

learn.save_encoder('ask-7-tuning-ask-twadr')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [77]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
print(labels)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

tensor([1676, 1321, 1204, 1204, 1227, 2048, 2048, 2048, 1669,  224,  224,  588,
        2147, 2149,  233, 2147,  249, 1204,  301, 1974, 1356, 1099,  725, 1711,
         445, 2149, 1054,  791, 1054,  792, 1227,  445, 1308,  119, 1884,  974,
        1580,  852, 2023, 1204,  945,  974,  974,  974,  974,  974,  119,  119,
         912, 1019, 1227, 1062, 1718,  974,  974, 2032, 2149, 1136,  945, 1594,
        1798, 2104, 1973, 1227, 2066, 1295, 1319, 2149, 1597, 1356, 1348,   84,
        1610, 1432, 2149,  825,  974, 1227, 1508, 1595, 1597, 1597, 1597, 1597,
        1936, 1580, 1610, 1610, 1610, 1526, 1526, 1526,  632, 1526, 1711, 1711,
        1753, 1718, 1767, 1580, 1767, 1288, 1669, 2149, 1075, 1767, 1767,  445,
        1787, 1787, 1787, 1787, 1521, 1718, 1718, 2149,  974, 2149, 1099, 1227,
        1942, 1099, 2024, 1787, 2032, 1787, 1787, 1427, 2103,  813,  233,  233,
         233,  233, 1727, 2149, 1711, 2194, 1529,  824, 1321, 1321])
142
142
[1676, 1321, 1204, 1204, 1227, 2048, 2048, 

In [78]:
correct_num = 0

for i in range(0, 142):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

44
0.30985915492957744
